<a href="https://colab.research.google.com/github/McKingN/PROJET_DDEFI_2025/blob/main/projet_data_ddefi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git config --global user.email "clabessim@gmail.com"
!git config --global user.name "McKingN"
!git clone https://github.com/Data-project-team-ecm/PROJET_DDEFI_2025.git


Cloning into 'PROJET_DDEFI_2025'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), done.
Resolving deltas: 100% (1/1), done.


In [1]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import yfinance as yf

def is_valid_ticker(ticker: str) -> bool:
    """
    Vérifie si un ticker est valide en récupérant son historique
    via yfinance. Si l'historique est vide ou qu'une erreur survient,
    on considère le ticker comme invalide.
    """
    try:
        # On récupère l'historique sur 1 jour
        data = yf.download("AAPL", start="2024-01-01", end="2025-01-01")
        print(data)
        # Si le DataFrame est vide, on suppose que le ticker est invalide
        return not data.empty
    except Exception:
        print(Exception)
        return False

In [9]:
import os
import yfinance as yf
import pandas as pd

def telecharger_donnees_en_parquet(ticker: str, dossier: str = "data") -> None:
    """
    Télécharge toutes les données disponibles pour un ticker via yfinance
    et les enregistre sous forme de fichier parquet dans le dossier spécifié.
    """
    # Création du dossier s'il n'existe pas
    os.makedirs(dossier, exist_ok=True)
    
    # Chemin de fichier de sortie
    chemin_fichier = os.path.join(dossier, f"{ticker}.parquet")
    
    # Téléchargement des données depuis la première date disponible
    df = yf.download(ticker, period="max", interval="1d")
    
    if not df.empty:
        # Enregistrement au format Parquet
        df.to_parquet(chemin_fichier)
        print(f"Fichier parquet créé pour {ticker} : {chemin_fichier}")
    else:
        print(f"Aucune donnée téléchargée pour {ticker}.")

def telecharger_liste_tickers_en_parquet(tickers: list, dossier: str = "data") -> None:
    """
    Prend une liste de tickers et crée les fichiers parquet pour chacun
    en utilisant la fonction de téléchargement.
    """
    for t in tickers:
        telecharger_donnees_en_parquet(t, dossier=dossier)

def afficher_donnees_ticker(ticker: str, dossier: str = "data") -> pd.DataFrame:
    """
    Affiche (retourne) un DataFrame avec les données d'un ticker.
    1. Vérifie si le fichier parquet existe déjà dans le dossier.
    2. S'il n'existe pas, télécharge les données et les enregistre.
    3. Charge et retourne le DataFrame.
    """
    chemin_fichier = os.path.join(dossier, f"{ticker}.parquet")
    
    # Vérification de l'existence du fichier parquet
    if not os.path.exists(chemin_fichier):
        print(f"Le fichier {chemin_fichier} n'existe pas. Téléchargement des données...")
        telecharger_donnees_en_parquet(ticker, dossier=dossier)
    
    # Lecture des données depuis le parquet
    if os.path.exists(chemin_fichier):
        df = pd.read_parquet(chemin_fichier)
        print(f"Données pour {ticker} :")
        print(df.head())  # Affichage (optionnel : on peut retourner le df directement)
        return df
    else:
        print(f"Impossible d'afficher les données : aucune donnée n'a été trouvée ou téléchargée pour {ticker}.")
        return pd.DataFrame()  # Retourne un DataFrame vide si échec


In [3]:
is_valid_ticker("AAPL")

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


Empty DataFrame
Columns: [(Adj Close, AAPL), (Close, AAPL), (High, AAPL), (Low, AAPL), (Open, AAPL), (Volume, AAPL)]
Index: []


False